In [4]:
import numpy as np
import pandas as pd
import os
import glob
from numpy import asarray, save, load
from sklearn.model_selection import train_test_split
from matplotlib.pyplot import imshow
from PIL import Image
import matplotlib.pyplot as plt
import shutil
from shutil import copy
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from timeit import default_timer as timer
%matplotlib inline

# Mount ADLS

In [5]:
def mountADLS():
    # azureml-core of version 1.0.72 or higher is required
    from azureml.core import Workspace, Dataset

    subscription_id = 'xxxxxxxxxx-xxxx-xxxx-xxxxxxxxx'
    resource_group = 'Your Azure RG'
    workspace_name = 'Your Azure ML Workspace Name'
    dataset_name = 'Name of your dataset in Azure ML'


    workspace = Workspace(subscription_id, resource_group, workspace_name)

    dataset = Dataset.get_by_name(workspace, name=dataset_name)

    # Create mountcontext and mount the dataset
    mount_ctx = dataset.mount()  
    mount_ctx.start()  

    # Get the mount point
    dataset_mount_folder = mount_ctx.mount_point
    print(dataset_mount_folder)

    # List the files in the mount point
    files = os.listdir(dataset_mount_folder)
    print(files)
    return dataset_mount_folder
    

In [6]:
mount = mountADLS()

/tmp/tmp53890s9n
["Phil's Upload", "Robin's Upload", 'TrainTestSetsBalanced', 'TrainTestSetsBalancedGreyscale', 'balancedSets', 'balancedSetsGreyscale', 'processed']


In [7]:
root=mount+'/processed/arrays'

# Find NumPy Files

In [8]:
fileList = []
for filename in os.listdir(root):
    f = os.path.join(root, filename)
    # checking if it is a file
    if os.path.isfile(f):
        fileList.append(f)


In [9]:
fileList

['/tmp/tmp53890s9n/processed/arrays/covidTestArray.npy',
 '/tmp/tmp53890s9n/processed/arrays/covidTestLabels.npy',
 '/tmp/tmp53890s9n/processed/arrays/covidTrainArray.npy',
 '/tmp/tmp53890s9n/processed/arrays/covidTrainLabels.npy',
 '/tmp/tmp53890s9n/processed/arrays/healthyTestArray.npy',
 '/tmp/tmp53890s9n/processed/arrays/healthyTestLabels.npy',
 '/tmp/tmp53890s9n/processed/arrays/healthyTrainArray.npy',
 '/tmp/tmp53890s9n/processed/arrays/healthyTrainLabels.npy',
 '/tmp/tmp53890s9n/processed/arrays/pneumoniaTestArray.npy',
 '/tmp/tmp53890s9n/processed/arrays/pneumoniaTestLabels.npy',
 '/tmp/tmp53890s9n/processed/arrays/pneumoniaTrainArray.npy',
 '/tmp/tmp53890s9n/processed/arrays/pneumoniaTrainLabels.npy']

In [10]:
covidList=[]
healthyList=[]
pneumoniaList=[]

for i in fileList:
    if 'covid' in i:
        covidList.append(i)
    elif 'healthy' in i:
        healthyList.append(i)
    else:
        pneumoniaList.append(i)
    


In [11]:
covidList

['/tmp/tmp53890s9n/processed/arrays/covidTestArray.npy',
 '/tmp/tmp53890s9n/processed/arrays/covidTestLabels.npy',
 '/tmp/tmp53890s9n/processed/arrays/covidTrainArray.npy',
 '/tmp/tmp53890s9n/processed/arrays/covidTrainLabels.npy']

In [12]:
covidTrainList=[]
covidTestList=[]
healthyTrainList=[]
healthyTestList=[]
pneumoniaTrainList=[]
pneumoniaTestList=[]


for i in covidList:
    if 'Train' in i:
        covidTrainList.append(i)
    elif "Test" in i:
        covidTestList.append(i)

for i in healthyList:
    if 'Train' in i:
        healthyTrainList.append(i)
    elif "Test" in i:
        healthyTestList.append(i)


for i in pneumoniaList:
    if 'Train' in i:
        pneumoniaTrainList.append(i)
    elif "Test" in i:
        pneumoniaTestList.append(i)

        



In [17]:
test = np.load(covidTrainList[0])

In [13]:
for i in covidTrainList:
    print(np.load(i))

[[[[0.3372549  0.3372549  0.3372549 ]
   [0.34509805 0.34509805 0.34509805]
   [0.34509805 0.34509805 0.34509805]
   ...
   [0.49803922 0.49803922 0.49803922]
   [0.5019608  0.5019608  0.5019608 ]
   [0.4627451  0.4627451  0.4627451 ]]

  [[0.32156864 0.32156864 0.32156864]
   [0.38039216 0.38039216 0.38039216]
   [0.3529412  0.3529412  0.3529412 ]
   ...
   [0.45490196 0.45490196 0.45490196]
   [0.4509804  0.4509804  0.4509804 ]
   [0.48235294 0.48235294 0.48235294]]

  [[0.4        0.4        0.4       ]
   [0.40392157 0.40392157 0.40392157]
   [0.36862746 0.36862746 0.36862746]
   ...
   [0.47058824 0.47058824 0.47058824]
   [0.44313726 0.44313726 0.44313726]
   [0.49411765 0.49411765 0.49411765]]

  ...

  [[0.7647059  0.7647059  0.7647059 ]
   [0.7647059  0.7647059  0.7647059 ]
   [0.7921569  0.7921569  0.7921569 ]
   ...
   [0.7764706  0.7764706  0.7764706 ]
   [0.8117647  0.8117647  0.8117647 ]
   [0.84705883 0.84705883 0.84705883]]

  [[0.75686276 0.75686276 0.75686276]
   [0.7

In [ ]:
cimages = np.load(covidTrainList[0])
plt.imshow(cimages[0])

# Unison Shuffle Copies

In [34]:
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

In [43]:
trainArrayList=[]
trainLabelList=[]
testArrayList=[]
testLabelList=[]
for i in fileList:
    if 'TrainArray' in i:
        trainArrayList.append(i)
    elif 'TrainLabel' in i:
        trainLabelList.append(i)
    elif 'TestArray' in i:
        testArrayList.append(i)
    elif 'TestLabel' in i:
        testLabelList.append(i)

In [44]:
print(trainArrayList)

['/tmp/tmproh_1s2y/processed/arrays/covidTrainArray.npy', '/tmp/tmproh_1s2y/processed/arrays/healthyTrainArray.npy', '/tmp/tmproh_1s2y/processed/arrays/pneumoniaTrainArray.npy']


In [22]:
x=np.load(trainArrayList[0]) #COVID
y=np.load(trainArrayList[1]) #healthy
z=np.load(trainArrayList[2]) #influenza
trainingset=np.concatenate([x,y,z])

In [23]:
a=np.load(trainLabelList[0]) #covid
b=np.load(trainLabelList[1]) #healthy
c=np.load(trainLabelList[2]) #influenza
trainingsetLabels=np.concatenate([a,b,c])

In [24]:
x=np.load(testArrayList[0])
y=np.load(testArrayList[1])
z=np.load(testArrayList[2])
testset=np.concatenate([x,y,z])

a=np.load(testLabelList[0])
b=np.load(testLabelList[1])
c=np.load(testLabelList[2])
testsetLabels=np.concatenate([a,b,c])

In [25]:
unison_shuffled_copies(trainingset,trainingsetLabels)

(array([[[[0.38431373, 0.38431373, 0.38431373],
          [0.3882353 , 0.3882353 , 0.3882353 ],
          [0.3882353 , 0.3882353 , 0.3882353 ],
          ...,
          [0.24313726, 0.24313726, 0.24313726],
          [0.24705882, 0.24705882, 0.24705882],
          [0.23921569, 0.23921569, 0.23921569]],
 
         [[0.39215687, 0.39215687, 0.39215687],
          [0.4       , 0.4       , 0.4       ],
          [0.4117647 , 0.4117647 , 0.4117647 ],
          ...,
          [0.27058825, 0.27058825, 0.27058825],
          [0.2627451 , 0.2627451 , 0.2627451 ],
          [0.25490198, 0.25490198, 0.25490198]],
 
         [[0.39607844, 0.39607844, 0.39607844],
          [0.40784314, 0.40784314, 0.40784314],
          [0.41568628, 0.41568628, 0.41568628],
          ...,
          [0.28235295, 0.28235295, 0.28235295],
          [0.26666668, 0.26666668, 0.26666668],
          [0.26666668, 0.26666668, 0.26666668]],
 
         ...,
 
         [[0.15686275, 0.15686275, 0.15686275],
          [0.15294

In [26]:
unison_shuffled_copies(testset,testsetLabels)

(array([[[[0.00392157, 0.00392157, 0.00392157],
          [0.00392157, 0.00392157, 0.00392157],
          [0.00392157, 0.00392157, 0.00392157],
          ...,
          [0.        , 0.        , 0.        ],
          [0.        , 0.        , 0.        ],
          [0.07450981, 0.07450981, 0.07450981]],
 
         [[0.00392157, 0.00392157, 0.00392157],
          [0.00392157, 0.00392157, 0.00392157],
          [0.00392157, 0.00392157, 0.00392157],
          ...,
          [0.00784314, 0.00784314, 0.00784314],
          [0.03921569, 0.03921569, 0.03921569],
          [0.10588235, 0.10588235, 0.10588235]],
 
         [[0.00392157, 0.00392157, 0.00392157],
          [0.        , 0.        , 0.        ],
          [0.        , 0.        , 0.        ],
          ...,
          [0.00392157, 0.00392157, 0.00392157],
          [0.23137255, 0.23137255, 0.23137255],
          [0.        , 0.        , 0.        ]],
 
         ...,
 
         [[0.79607844, 0.79607844, 0.79607844],
          [0.76078

In [28]:
np.save('X_train',trainingset)
np.save('y_train',trainingsetLabels)
np.save('X_test',testset)
np.save('y_test',testsetLabels)

# Upload Shuffled Copies to ADLS

In [ ]:
from azure.storage.blob import BlobClient

storage_account_key = input("input storage account key")

storage_url = "https://{storageaccountname}.blob.core.windows.net/" 

arraynames = []
for f in os.listdir():
    if '.npy' in f:
        arraynames.append(f)
        
for a in arraynames:
    blob_client = BlobClient(storage_url, container_name="images/processed/grouped", blob_name=a, credential=storage_account_key)
    with open(a, "rb") as data:
        blob_client.upload_blob(data)
